Lab 1 - Geometric primitives and transformations


(dịch chuyển, xoay, co giãn)


In [4]:
!pip install opencv-python
!pip install matplotlib


In [ ]:
import numpy as np
import cv2

# Global state
drawing = False
rect_start = (-1, -1)
rect_end = (-1, -1)
image = np.ones((720, 1280, 3), dtype=np.uint8) * 255
image_backup = image.copy()
rectangle_drawn = False

# Helper: draw grid


def draw_grid(img, step=100):
    for x in range(0, img.shape[1], step):
        cv2.line(img, (x, 0), (x, img.shape[0]), (200, 200, 200), 1)
        cv2.putText(img, f"{x}", (x+2, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 100, 100), 1)
    for y in range(0, img.shape[0], step):
        cv2.line(img, (0, y), (img.shape[1], y), (200, 200, 200), 1)
        cv2.putText(img, f"{y}", (2, y-2),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 100, 100), 1)

# Mouse event handler


def draw_rectangle(event, x, y, flags, param):
    global drawing, rect_start, rect_end, image, image_backup, rectangle_drawn

    if rectangle_drawn:
        return

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        rect_start = (x, y)

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        temp = image.copy()
        cv2.rectangle(temp, rect_start, (x, y), (0, 0, 255), 2)
        draw_grid(temp)
        cv2.imshow("Draw Rectangle", temp)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        rect_end = (x, y)
        rectangle_drawn = True
        cv2.rectangle(image, rect_start, rect_end, (0, 0, 255), 2)
        cv2.putText(image, f"P1{rect_start}", (rect_start[0]+5, rect_start[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        cv2.putText(image, f"P2{rect_end}", (rect_end[0]+5, rect_end[1]+20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        image_backup = image.copy()
        draw_grid(image)
        cv2.imshow("Draw Rectangle", image)

# Transformations


def apply_translation(img, start, end, dx, dy):
    trans_start = (start[0] + dx, start[1] + dy)
    trans_end = (end[0] + dx, end[1] + dy)
    cv2.rectangle(img, trans_start, trans_end, (0, 255, 0), 2)
    cv2.putText(img, f"T1{trans_start}", (trans_start[0]+5, trans_start[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 128, 0), 2)
    cv2.putText(img, f"T2{trans_end}", (trans_end[0]+5, trans_end[1]+20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 128, 0), 2)


def apply_rotation(img, start, end, angle_deg):
    cx = (start[0] + end[0]) / 2
    cy = (start[1] + end[1]) / 2
    angle = np.radians(angle_deg)

    corners = np.array([
        [start[0], start[1]],
        [end[0], start[1]],
        [end[0], end[1]],
        [start[0], end[1]]
    ])

    R = np.array([
        [np.cos(angle), -np.sin(angle)],
        [np.sin(angle),  np.cos(angle)]
    ])

    rotated = []
    for pt in corners:
        shifted = pt - [cx, cy]
        rotated_pt = R @ shifted + [cx, cy]
        rotated.append(rotated_pt)
    rotated = np.int32(rotated)

    cv2.polylines(img, [rotated], True, (255, 0, 0), 2)
    cv2.putText(img, f"R1{tuple(rotated[0])}", rotated[0] + [0, -10],
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    cv2.putText(img, f"R3{tuple(rotated[2])}", rotated[2] + [0, 20],
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)


def apply_scaling(img, start, end, sx, sy):
    cx = (start[0] + end[0]) / 2
    cy = (start[1] + end[1]) / 2

    corners = np.array([
        [start[0], start[1]],
        [end[0], start[1]],
        [end[0], end[1]],
        [start[0], end[1]]
    ])

    scaled = []
    for pt in corners:
        shifted = pt - [cx, cy]
        scaled_pt = shifted * [sx, sy] + [cx, cy]
        scaled.append(scaled_pt)
    scaled = np.int32(scaled)

    cv2.polylines(img, [scaled], True, (0, 0, 0), 2)
    cv2.putText(img, f"S1{tuple(scaled[0])}", scaled[0] + [0, -10],
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
    cv2.putText(img, f"S3{tuple(scaled[2])}", scaled[2] + [0, 20],
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)


# Run GUI
cv2.namedWindow("Draw Rectangle")
cv2.setMouseCallback("Draw Rectangle", draw_rectangle)
draw_grid(image)
cv2.imshow("Draw Rectangle", image)

cv2.waitKey(0)
cv2.destroyAllWindows()


Ví dụ: Thầy chạy từng phần dưới để xem kết quả nhé!


Ghi chú tọa độ:

T1, T2 là 2 điểm sau khi dịch chuyển

R1, R3 là 2 điểm sau khi xoay (đường chéo)

S1, S3 là 2 điểm sau khi co giãn


In [ ]:
# Di chuyển 100px sang phải và 50px xuống dưới
img2 = image_backup.copy()
apply_translation(img2, rect_start, rect_end, 100, 50)
draw_grid(img2)
cv2.imshow("Draw Rectangle", img2)
cv2.waitKey(0)


-1

In [ ]:
# Xoay 30 độ:
img2 = image_backup.copy()
apply_rotation(img2, rect_start, rect_end, 30)
draw_grid(img2)
cv2.imshow("Draw Rectangle", img2)
cv2.waitKey(0)


-1

In [ ]:
# Tỷ lệ 1,5x theo chiều ngang, 0,5x theo chiều dọc:
img2 = image_backup.copy()
apply_scaling(img2, rect_start, rect_end, 1.5, 0.5)
draw_grid(img2)
cv2.imshow("Draw Rectangle", img2)
cv2.waitKey(0)


-1

Test thêm 2 trường hợp Affine Transformation & Projective Transform


In [ ]:
import numpy as np
import cv2

# Affine


def apply_affine_transform(img, start, end):
    # Gốc hình chữ nhật
    p1 = np.float32([start, (end[0], start[1]), end])
    # Tạo hình méo nhẹ
    p2 = np.float32([
        (start[0] + 30, start[1] + 20),
        (end[0] - 30, start[1] + 50),
        (end[0], end[1] + 10)
    ])
    M = cv2.getAffineTransform(p1, p2)
    rows, cols = img.shape[:2]
    transformed = cv2.warpAffine(img, M, (cols, rows))

    cv2.putText(transformed, "Affine Transform", (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (128, 0, 255), 2)
    return transformed

# Projective


def apply_projective_transform(img, start, end):
    pts1 = np.float32([
        start,
        (end[0], start[1]),
        end,
        (start[0], end[1])
    ])
    pts2 = np.float32([
        (start[0] + 60, start[1] + 20),
        (end[0] - 60, start[1] + 60),
        (end[0], end[1]),
        (start[0], end[1] - 50)
    ])
    M = cv2.getPerspectiveTransform(pts1, pts2)
    rows, cols = img.shape[:2]
    transformed = cv2.warpPerspective(img, M, (cols, rows))

    cv2.putText(transformed, "Projective Transform", (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 128, 255), 2)
    return transformed


# Tạo hình chữ nhật mẫu để test
img_test = np.ones((720, 1280, 3), dtype=np.uint8) * 255
start_test = (300, 200)
end_test = (500, 400)
cv2.rectangle(img_test, start_test, end_test, (0, 0, 255), 2)
cv2.putText(img_test, f"P1{start_test}", (start_test[0]+5, start_test[1]-10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
cv2.putText(img_test, f"P2{end_test}", (end_test[0]+5, end_test[1]+20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

# Áp dụng
affine_img = apply_affine_transform(img_test, start_test, end_test)
projective_img = apply_projective_transform(img_test, start_test, end_test)

# Hiển thị
cv2.imshow("Original Rectangle", img_test)
cv2.imshow("Affine Transform", affine_img)
cv2.imshow("Projective Transform", projective_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
